# Import Libraries

In [1]:
import matplotlib.pyplot as plt 
import pandas as pd
import data_preprocess
import boto3, re
import sagemaker
from sagemaker import get_execution_role
import os
import numpy as np

sagemaker_session = sagemaker.Session()
role = get_execution_role()
!mkdir data



mkdir: cannot create directory ‘data’: File exists


# Get CSV data

In [2]:
telemetry = pd.read_csv('data/PdM_telemetry.csv')
errors = pd.read_csv('data/PdM_errors.csv')
maint = pd.read_csv('data/PdM_maint.csv')
failures = pd.read_csv('data/PdM_failures.csv')
machines = pd.read_csv('data/PdM_machines.csv')

# Telemetry Preprocess

In [3]:
tele = data_preprocess.telemetry_preprocess(telemetry)
print(tele)

Total number of telemetry records: 876100
                  datetime  machineID        volt      rotate    pressure  \
0      2015-01-01 06:00:00          1  176.217853  418.504078  113.077935   
1      2015-01-01 07:00:00          1  162.879223  402.747490   95.460525   
2      2015-01-01 08:00:00          1  170.989902  527.349825   75.237905   
3      2015-01-01 09:00:00          1  162.462833  346.149335  109.248561   
4      2015-01-01 10:00:00          1  157.610021  435.376873  111.886648   
...                    ...        ...         ...         ...         ...   
876095 2016-01-01 02:00:00        100  179.438162  395.222827  102.290715   
876096 2016-01-01 03:00:00        100  189.617555  446.207972   98.180607   
876097 2016-01-01 04:00:00        100  192.483414  447.816524   94.132837   
876098 2016-01-01 05:00:00        100  165.475310  413.771670  104.081073   
876099 2016-01-01 06:00:00        100  171.336037  496.096870   79.095538   

        vibration  
0       45.08

# Errors Preprocess

In [4]:
err = data_preprocess.error_preprocess(errors)
print(err)

Total Number of error records: 3919
                datetime  machineID errorID
0    2015-01-03 07:00:00          1  error1
1    2015-01-03 20:00:00          1  error3
2    2015-01-04 06:00:00          1  error5
3    2015-01-10 15:00:00          1  error4
4    2015-01-22 10:00:00          1  error4
...                  ...        ...     ...
3914 2015-11-21 08:00:00        100  error2
3915 2015-12-04 02:00:00        100  error1
3916 2015-12-08 06:00:00        100  error2
3917 2015-12-08 06:00:00        100  error3
3918 2015-12-22 03:00:00        100  error3

[3919 rows x 3 columns]


# Maintenance Preprocess

In [5]:
maint = data_preprocess.maint_preprocess(maint)
print(maint)

Total Number of maintenance Records: 3286
                datetime  machineID   comp
0    2014-06-01 06:00:00          1  comp2
1    2014-07-16 06:00:00          1  comp4
2    2014-07-31 06:00:00          1  comp3
3    2014-12-13 06:00:00          1  comp1
4    2015-01-05 06:00:00          1  comp4
...                  ...        ...    ...
3281 2015-10-10 06:00:00        100  comp3
3282 2015-10-25 06:00:00        100  comp4
3283 2015-11-09 06:00:00        100  comp4
3284 2015-12-09 06:00:00        100  comp2
3285 2015-12-24 06:00:00        100  comp2

[3286 rows x 3 columns]


# Machine Preprocess

In [6]:
mach = data_preprocess.mach_preprocess(machines)
print(mach)

Total number of machines: 100
    machineID   model  age
0           1  model3   18
1           2  model4    7
2           3  model3    8
3           4  model3    7
4           5  model3    2
..        ...     ...  ...
95         96  model2   10
96         97  model2   14
97         98  model2   20
98         99  model1   14
99        100  model4    5

[100 rows x 3 columns]


# Failure Preprocess

In [7]:
failure = data_preprocess.failure_preprocess(failures)
print(failure)

Total number of failures: 761
               datetime  machineID failure
0   2015-01-05 06:00:00          1   comp4
1   2015-03-06 06:00:00          1   comp1
2   2015-04-20 06:00:00          1   comp2
3   2015-06-19 06:00:00          1   comp4
4   2015-09-02 06:00:00          1   comp4
..                  ...        ...     ...
756 2015-11-29 06:00:00         99   comp3
757 2015-12-14 06:00:00         99   comp4
758 2015-02-12 06:00:00        100   comp1
759 2015-09-10 06:00:00        100   comp1
760 2015-12-09 06:00:00        100   comp2

[761 rows x 3 columns]


# Getting Lag Features from telemetry data

In [8]:
lag_tele = data_preprocess.get_lagfeatures_tele(tele)
print(lag_tele.describe())

        machineID            datetime  voltmean_3h  rotatemean_3h  \
0               1 2015-01-01 09:00:00   170.028993     449.533798   
1               1 2015-01-01 12:00:00   164.192565     403.949857   
2               1 2015-01-01 15:00:00   168.134445     435.781707   
3               1 2015-01-01 18:00:00   165.514453     430.472823   
4               1 2015-01-01 21:00:00   168.809347     437.111120   
...           ...                 ...          ...            ...   
292095        100 2015-12-31 21:00:00   166.078945     424.431184   
292096        100 2016-01-01 00:00:00   158.327984     504.056262   
292097        100 2016-01-01 03:00:00   170.306956     438.653616   
292098        100 2016-01-01 06:00:00   182.525427     435.932056   
292099        100 2016-01-01 09:00:00   171.336037     496.096870   

        pressuremean_3h  vibrationmean_3h  
0             94.592122         40.893502  
1            105.687417         34.255891  
2            107.793709         41.2394

# Getting Lag Features from error data

In [9]:
lag_err  = data_preprocess.get_lagfeatures_error(err, tele)
print(lag_err.describe())

(3919, 7)
          machineID    error1count    error2count    error3count  \
count  291400.00000  291400.000000  291400.000000  291400.000000   
mean       50.50000       0.027649       0.027069       0.022907   
std        28.86612       0.166273       0.164429       0.151453   
min         1.00000       0.000000       0.000000       0.000000   
25%        25.75000       0.000000       0.000000       0.000000   
50%        50.50000       0.000000       0.000000       0.000000   
75%        75.25000       0.000000       0.000000       0.000000   
max       100.00000       2.000000       2.000000       2.000000   

         error4count    error5count  
count  291400.000000  291400.000000  
mean        0.019904       0.009753  
std         0.140820       0.098797  
min         0.000000       0.000000  
25%         0.000000       0.000000  
50%         0.000000       0.000000  
75%         0.000000       0.000000  
max         2.000000       2.000000  


# Component Repair Data

In [10]:
comp_rep = data_preprocess.component_rep(tele, maint)
print(comp_rep.describe())

           machineID          comp1          comp2          comp3  \
count  876100.000000  876100.000000  876100.000000  876100.000000   
mean       50.500000      53.525185      51.540806      52.725962   
std        28.866087      62.491679      59.269254      58.873114   
min         1.000000       0.000000       0.000000       0.000000   
25%        25.750000      13.291667      12.125000      13.125000   
50%        50.500000      32.791667      29.666667      32.291667   
75%        75.250000      68.708333      66.541667      67.333333   
max       100.000000     491.958333     348.958333     370.958333   

               comp4  
count  876100.000000  
mean       53.834191  
std        59.707978  
min         0.000000  
25%        13.000000  
50%        32.500000  
75%        70.458333  
max       394.958333  


/home/ec2-user/SageMaker/Predictive_Maintenance/Component_Failure_Prediction_Tensorflow/data_preprocess.py:179: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  comp_rep[comp] = (comp_rep['datetime'] - comp_rep[comp].values.astype('datetime64[ns]')) / np.timedelta64(1, 'D')


# Final Features Generation

In [11]:
final_feat = data_preprocess.get_final_features(lag_tele, lag_err, comp_rep, mach)
print(final_feat.describe())

          machineID    voltmean_3h  rotatemean_3h  pressuremean_3h  \
count  291300.00000  291300.000000  291300.000000    291300.000000   
mean       50.50000     170.771416     446.612973       100.855301   
std        28.86612       9.497201      33.122865         7.408854   
min         1.00000     125.532506     211.811184        72.118639   
25%        25.75000     164.445576     427.568132        96.238173   
50%        50.50000     170.430790     448.385564       100.233139   
75%        75.25000     176.607051     468.447680       104.404520   
max       100.00000     241.420717     586.682904       162.309656   

       vibrationmean_3h      voltsd_3h    rotatesd_3h  pressuresd_3h  \
count     291300.000000  291300.000000  291300.000000  291300.000000   
mean          40.384328      13.299858      44.456335       8.885822   
std            3.476641       6.966647      23.216421       4.656341   
min           26.569635       0.025509       0.078991       0.027417   
25%      

# Label Construction

In [12]:
labeled_feat = data_preprocess.label_construct(final_feat, failure)

#cond = labeled_feat.loc[(labeled_feat['datetime'] > pd.to_datetime('2015-08-01 01:00:00')) & (labeled_feat['datetime']  < pd.to_datetime('2015-11-30 01:00:00'))]
#cond =  labeled_feat.query('datetime> 2015-08-01 01:00:00 & datetime< 2015-11-30 01:00:00')    
#print(cond)



print(labeled_feat.columns)

Index(['machineID', 'datetime', 'voltmean_3h', 'rotatemean_3h',
       'pressuremean_3h', 'vibrationmean_3h', 'voltsd_3h', 'rotatesd_3h',
       'pressuresd_3h', 'vibrationsd_3h', 'voltmean_24h', 'rotatemean_24h',
       'pressuremean_24h', 'vibrationmean_24h', 'voltstd_24h', 'rotatestd_24h',
       'pressurestd_24h', 'vibrationstd_24h', 'error1count', 'error2count',
       'error3count', 'error4count', 'error5count', 'comp1', 'comp2', 'comp3',
       'comp4', 'model', 'age', 'failure'],
      dtype='object')


# Training and testing data

In [13]:
import utils

dataset = utils.data_preprocess_func(labeled_feat)
print(dataset)



train_y 0         4
1         4
2         4
3         4
4         4
         ..
290102    4
290103    4
290104    4
290105    4
290106    4
Name: failure, Length: 167922, dtype: int64
train_X         machineID  voltmean_3h  rotatemean_3h  pressuremean_3h  \
0               1   180.133784     440.608320        94.137969   
1               1   176.364293     439.349655       101.553209   
2               1   160.384568     424.385316        99.598722   
3               1   170.472461     442.933997       102.380586   
4               1   163.263806     468.937558       102.726648   
...           ...          ...            ...              ...   
290102        100   181.789600     443.816655        95.819894   
290103        100   173.083263     438.704284        94.424390   
290104        100   174.048390     433.016353        99.925802   
290105        100   167.883990     435.511531        96.602542   
290106        100   176.781659     442.259006        97.540743   

        vibrat

test_X         machineID  voltmean_3h  rotatemean_3h  pressuremean_3h  \
2663            1   163.987293     437.978326       102.948709   
2664            1   187.332533     484.125338       107.210719   
2665            1   174.786762     447.158129        95.172683   
2666            1   164.980428     520.921153       112.973879   
2667            1   176.934686     440.510459       124.692271   
...           ...          ...            ...              ...   
291336        100   177.758754     458.391629       103.183160   
291337        100   166.078945     424.431184        98.344409   
291338        100   158.327984     504.056262       108.055441   
291339        100   170.306956     438.653616       102.346227   
291340        100   182.525427     435.932056        98.798172   

        vibrationmean_3h  voltsd_3h  rotatesd_3h  pressuresd_3h  \
2663           38.472804  18.422113    58.675392       6.656489   
2664           40.659036  12.039940    34.112798       4.061496   

In [14]:
print(dataset['test_dict']['features'].shape)

(25003, 31)


In [15]:
!mkdir tfrecord_data
utils.convert_to(dataset["train_dict"], 'train', 'tfrecord_data')
utils.convert_to(dataset["val_dict"], 'test', 'tfrecord_data')

mkdir: cannot create directory ‘tfrecord_data’: File exists
Writing tfrecord_data/train.tfrecords
Writing tfrecord_data/test.tfrecords


In [16]:
bucket = "tensorflow-component-failure"
inputs = sagemaker_session.upload_data(path='tfrecord_data',bucket=bucket, key_prefix='Tfrecord_data')

# Training the model

In [17]:
from sagemaker.tensorflow import TensorFlow
output_path = "s3://tensorflow-component-failure/model"

comp_fail_estimator = TensorFlow(entry_point='component_classifier.py',
                             role=role,
                             framework_version='1.11.0',
                             output_path = output_path,
                             training_steps=1000, 
                             evaluation_steps=100,
                             #early_stopping="true",
                             #patience="5",
                             train_instance_count=1,
                             train_instance_type='ml.m4.xlarge')

comp_fail_estimator.fit(inputs)

2.1.0 is the latest version of tensorflow that supports Python 2. Newer versions of tensorflow will only be available for Python 3.Please set the argument "py_version='py3'" to use the Python 3 tensorflow image.
'create_image_uri' will be deprecated in favor of 'ImageURIProvider' class in SageMaker Python SDK v2.
's3_input' class will be renamed to 'TrainingInput' in SageMaker Python SDK v2.
'create_image_uri' will be deprecated in favor of 'ImageURIProvider' class in SageMaker Python SDK v2.


2020-10-14 09:50:19 Starting - Starting the training job...
2020-10-14 09:50:24 Starting - Launching requested ML instances......
2020-10-14 09:51:41 Starting - Preparing the instances for training......
2020-10-14 09:52:38 Downloading - Downloading input data
2020-10-14 09:52:38 Training - Downloading the training image..2020-10-14 09:52:58,700 INFO - root - running container entrypoint
2020-10-14 09:52:58,701 INFO - root - starting train task
2020-10-14 09:52:58,716 INFO - container_support.training - Training starting
2020-10-14 09:53:01,490 INFO - tf_container - ----------------------TF_CONFIG--------------------------
2020-10-14 09:53:01,490 INFO - tf_container - {"environment": "cloud", "cluster": {"master": ["algo-1:2222"]}, "task": {"index": 0, "type": "master"}}
2020-10-14 09:53:01,490 INFO - tf_container - ---------------------------------------------------------
2020-10-14 09:53:01,490 INFO - tf_container - creating RunConfig:
2020-10-14 09:53:01,491 INFO - tf_container - {'


2020-10-14 09:53:14 Uploading - Uploading generated training model2020-10-14 09:53:08,641 INFO - tensorflow - Running local_init_op.
2020-10-14 09:53:08,651 INFO - tensorflow - Done running local_init_op.
2020-10-14 09:53:08,751 INFO - tensorflow - Evaluation [10/100]
2020-10-14 09:53:08,765 INFO - tensorflow - Evaluation [20/100]
2020-10-14 09:53:08,779 INFO - tensorflow - Evaluation [30/100]
2020-10-14 09:53:08,794 INFO - tensorflow - Evaluation [40/100]
2020-10-14 09:53:08,809 INFO - tensorflow - Evaluation [50/100]
2020-10-14 09:53:08,824 INFO - tensorflow - Evaluation [60/100]
2020-10-14 09:53:08,838 INFO - tensorflow - Evaluation [70/100]
2020-10-14 09:53:08,853 INFO - tensorflow - Evaluation [80/100]
2020-10-14 09:53:08,868 INFO - tensorflow - Evaluation [90/100]
2020-10-14 09:53:08,883 INFO - tensorflow - Evaluation [100/100]
2020-10-14 09:53:08,911 INFO - tensorflow - Finished evaluation at 2020-10-14-09:53:08
2020-10-14 09:53:08,911 INFO - tensorflow - Saving dict for global


2020-10-14 09:53:21 Completed - Training job completed
Training seconds: 55
Billable seconds: 55


# Compiling the model for Raspberry Pi

In [24]:
output_path = "s3://tensorflow-component-failure/model"

print(output_path)


optimized_estimator = comp_fail_estimator.compile_model(target_instance_family='rasp3b',
                              output_path=output_path,
                              input_shape= {'data':[1,31]},
                              framework='tensorflow', framework_version='1.11.0')

Parameter image will be renamed to image_uri in SageMaker Python SDK v2.
2.1.0 is the latest version of tensorflow that supports Python 2. Newer versions of tensorflow will only be available for Python 3.Please set the argument "py_version='py3'" to use the Python 3 tensorflow image.
'create_image_uri' will be deprecated in favor of 'ImageURIProvider' class in SageMaker Python SDK v2.


s3://tensorflow-component-failure/model
?.....!

The instance type rasp3b is not supported for deployment via SageMaker.Please deploy the model manually.


# Deploy the uncompiled model on ml_m4 instance

In [19]:
comp_fail_predictor = comp_fail_estimator.deploy(initial_instance_count=1,instance_type='ml.m4.xlarge')

Parameter image will be renamed to image_uri in SageMaker Python SDK v2.
2.1.0 is the latest version of tensorflow that supports Python 2. Newer versions of tensorflow will only be available for Python 3.Please set the argument "py_version='py3'" to use the Python 3 tensorflow image.
'create_image_uri' will be deprecated in favor of 'ImageURIProvider' class in SageMaker Python SDK v2.


-----------!

# Prediction

In [20]:
from sagemaker.predictor import csv_serializer, json_deserializer

comp_fail_predictor.content_type = 'text/csv'
comp_fail_predictor.serializer = csv_serializer
comp_fail_predictor.deserializer = json_deserializer

In [27]:
test_features=dataset['test_dict']['features']
test_labels = dataset['test_dict']['labels']

from numpy import asarray
from numpy import save

data_features = asarray(dataset['test_dict']['features'])
save('data_features.npy', data_features)

data_labels = asarray(dataset['test_dict']['labels'])
save('data_labels.npy', data_labels)


# true_pred = 0
# false_pred = 0
# for tf in range(25003):
#     prediction = comp_fail_predictor.predict(test_features[tf])
#     #print(test_labels[tf],np.argmax(prediction['outputs']['probabilities']['floatVal']))
#     if(test_labels[tf] == np.argmax(prediction['outputs']['probabilities']['floatVal'])):
#         true_pred +=1
#     else:
#         false_pred+=1


In [23]:
print(true_pred/25003*100)
print(true_pred, false_pred)

98.16022077350718
24543 460


In [28]:
comp_fail_predictor.delete_endpoint()